<a href="https://colab.research.google.com/github/kikifana/Team_1_WeLead/blob/main/D03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_curve, auc, ConfusionMatrixDisplay
)
from sklearn.preprocessing import label_binarize
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_curve, auc, ConfusionMatrixDisplay
)
from sklearn.preprocessing import label_binarize
import os

# Create directory for saving plots if it doesn't exist
plot_dir = "classification_results"
os.makedirs(plot_dir, exist_ok=True)

def evaluate_and_plot_model(model, X_train, X_test, y_train, y_test, dataset_type):

    print(f"\n Training {model.__class__.__name__} on {dataset_type}...")
    model.fit(X_train, y_train)
    y_pred_test = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred_test)
    precision = precision_score(y_test, y_pred_test, average="weighted")
    recall = recall_score(y_test, y_pred_test, average="weighted")
    f1 = f1_score(y_test, y_pred_test, average="weighted")


    print(f"\n {model.__class__.__name__} Performance ({dataset_type}):")
    print(f" Accuracy:  {accuracy:.4f}")
    print(f" Precision: {precision:.4f}")
    print(f" Recall:    {recall:.4f}")
    print(f" F1 Score:  {f1:.4f}")


    cm = confusion_matrix(y_test, y_pred_test)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.unique(y_test))


    plt.figure(figsize=(6, 6))
    disp.plot(cmap="Blues", values_format=".0f")
    plt.title(f"Confusion Matrix - {model.__class__.__name__} ({dataset_type})")


    cm_path = os.path.join(plot_dir, f"confusion_matrix_{model.__class__.__name__}_{dataset_type}.png")
    plt.savefig(cm_path, dpi=300)
    print(f" Confusion matrix saved: {cm_path}")

    plt.show()


    return {
        "Model": model.__class__.__name__,
        "Dataset": dataset_type,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1 Score": f1
    }



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

models = {
    "RandomForest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(C=1, gamma=0.01, kernel='rbf', probability=True),
    "MLP": MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=600, random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "LogReg": LogisticRegression(max_iter=200, random_state=42)
}
results = []
datasets = {
    "Manual Clustering (Standard)": (X_train_manual_std, X_test_manual_std, y_train_manual_std, y_test_manual_std),
    "KMeans Clustering (Standard)": (X_train_kmeans_std, X_test_kmeans_std, y_train_kmeans_std, y_test_kmeans_std),
    "Manual Clustering (MinMax)": (X_train_manual_mm, X_test_manual_mm, y_train_manual_mm, y_test_manual_mm),
    "KMeans Clustering (MinMax)": (X_train_kmeans_mm, X_test_kmeans_mm, y_train_kmeans_mm, y_test_kmeans_mm)
}


for name, model in models.items():
    for dataset_type, (X_train, X_test, y_train, y_test) in datasets.items():
        result = evaluate_and_plot_model(model, X_train, X_test, y_train, y_test, dataset_type)
        results.append(result)


results_df = pd.DataFrame(results)
print(results_df)




In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
import pandas as pd
X = X_train_kmeans_std
y = y_train_kmeans_std

def evaluate_models_with_cv(models, X, y, cv_folds=5):

    skf = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
    results = []

    for name, model in models.items():
        print(f"\n Evaluating {name} with {cv_folds}-fold Cross-Validation...")
        accuracy = cross_val_score(model, X, y, cv=skf, scoring='accuracy').mean()
        precision = cross_val_score(model, X, y, cv=skf, scoring='precision_weighted').mean()
        recall = cross_val_score(model, X, y, cv=skf, scoring='recall_weighted').mean()
        f1 = cross_val_score(model, X, y, cv=skf, scoring='f1_weighted').mean()

        print(f" {name} Performance (Cross-Validation):")
        print(f" Accuracy:  {accuracy:.4f}")
        print(f" Precision: {precision:.4f}")
        print(f" Recall:    {recall:.4f}")
        print(f" F1 Score:  {f1:.4f}")

        results.append({
            "Model": name,
            "CV Accuracy": accuracy,
            "CV Precision": precision,
            "CV Recall": recall,
            "CV F1 Score": f1
        })

    return pd.DataFrame(results)

cv_results = evaluate_models_with_cv(models, X, y, cv_folds=5)

cv_results_df = pd.DataFrame(cv_results)
display(cv_results_df)


In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd

rf_selector = RandomForestClassifier(n_estimators=200, random_state=42)
rf_selector.fit(X_train_kmeans_std, y_train_kmeans_std)
feature_importance = rf_selector.feature_importances_
features = X_train_kmeans_std.columns
sorted_idx = np.argsort(feature_importance)[::-1]
top_features = [features[i] for i in sorted_idx[:5]]

plt.figure(figsize=(8, 4))
plt.barh(np.array(features)[sorted_idx][:5], feature_importance[sorted_idx][:5])
plt.xlabel("Feature Importance")
plt.title("Top 5 Important Features (Random Forest)")
plt.gca().invert_yaxis()
plt.show()

print(f"Selected Top 5 Features: {top_features}")

In [ ]:

X_top5 = X_train_kmeans_std[top_features]

X_train, X_test, y_train, y_test = train_test_split(X_top5, y_train_kmeans_std, test_size=0.2, random_state=42, stratify=y_train_kmeans_std)

rf_top5 = RandomForestClassifier(n_estimators=200, random_state=42)
rf_top5.fit(X_train, y_train)
y_pred_test = rf_top5.predict(X_test)

accuracy_top5 = accuracy_score(y_test, y_pred_test)
print(f" Random Forest Accuracy with Top 5 Features: {accuracy_top5:.4f}")